In [1]:
# -*- coding: utf-8 -*-
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, qApp
from ga import Ui_MainWindow
import pandas as pd
from PyQt5.QtCore import QCoreApplication
from PyQt5 import QtWidgets, QtGui
from PyQt5.QtWidgets import QFileDialog

class MainWindow(QMainWindow, Ui_MainWindow):
    def __init__(self, parent=None):    
        super(MainWindow, self).__init__(parent)
        self.setupUi(self)
        self.setWindowIcon(QtGui.QIcon('logo.ico'))
        self.actionExit.triggered.connect(self.close)
        self.actionOpen.triggered.connect(self.openxlsx)
        self.statusBar().showMessage('Ready')
        
    def openxlsx(self):
        fileName, filetype = QFileDialog.getOpenFileName(self,
        "選取檔案",
        "./",
        "All Files (*);;Text Files (*.xlsx)")
        read = pd.read_excel(fileName)
        self.article={}
        for i in range(len(read)):
            self.article[i+1]=list(read.loc[i].values)
        self.textBrowser_show.append(str(read.values))
        
    def inputdata(self):
        read = pd.read_excel("data.xlsx")
        self.textBrowser_show.append(str(read.values))
        
    def run(self):
        import random
        import numpy as np
        import pandas as pd
   
        population_size = eval(self.lineEdit_population.text())  #族群大小
        selection_rate = eval(self.lineEdit_selection.text()) #選擇率
        crossover_rate = eval(self.lineEdit_crossover.text())  #交配率
        mutation_rate = eval(self.lineEdit_mutation.text())  #突變率
        epoch = eval(self.lineEdit_epoch.text())     

        def fitness_function(chromosome=None, max_weight=150, data=None, penalty_value=5):
            """輸入一組染色體求適應值"""
            fitness_value = 0 #適應值 
            weight = 0        #重量
            for i, c in enumerate(chromosome):
                if chromosome[i]:
                    fitness_value += data[i+1][0]
                    weight += data[i+1][1]   
            if weight >max_weight:           #懲罰
                fitness_value = penalty_value
            return fitness_value

        def Roulette(data=None):
            """輸入族群給出輪盤選中的個體"""
            import random
            import numpy as np
            r = random.uniform(0, 1)
            for i in range(len(np.array(data))):
                if sum((np.array(data)/sum(np.array(data)))[0:i+1]) > r:
                    break
            return i

        def crossover(chromosome_1, chromosome_2, point=2):
            """輸入兩組染色體做交配"""
            p = random.sample(range(len(chromosome_1)), k=point)
            new_chromosome_1 = []
            new_chromosome_2 = []
            for i in range(len(chromosome_1)):
                if i in p:
                    new_chromosome_1.append(chromosome_2[i])
                    new_chromosome_2.append(chromosome_1[i])
                else:
                    new_chromosome_1.append(chromosome_1[i])
                    new_chromosome_2.append(chromosome_2[i])

            return new_chromosome_1, new_chromosome_2

        def mutation(chromosome, point=1):
            """輸入一組染色體做突變"""
            p = random.sample(range(len(chromosome)), k=point)
            new_chromosome = chromosome
            for i in p:
                if chromosome[i] == 0:
                    new_chromosome[i] = 1
                else:
                    new_chromosome[i] = 0
            return new_chromosome

        chromosome_0 = np.array([random.randint(0, 1) for _ in range(population_size)])
        chromosome_1 = np.array([random.randint(0, 1) for _ in range(population_size)])
        chromosome_2 = np.array([random.randint(0, 1) for _ in range(population_size)])
        chromosome_3 = np.array([random.randint(0, 1) for _ in range(population_size)])
        chromosome_4 = np.array([random.randint(0, 1) for _ in range(population_size)])
        chromosome_5 = np.array([random.randint(0, 1) for _ in range(population_size)])
        chromosome_6 = np.array([random.randint(0, 1) for _ in range(population_size)])

        population = np.stack((chromosome_0, chromosome_1, chromosome_2, chromosome_3, chromosome_4, chromosome_5,  chromosome_6), axis=-1)
        population  #初始族群
        
        counter = 0
        while counter<epoch:
            value = []
            for p in population:
                value.append(fitness_function(chromosome=p, max_weight=150, data=self.article, penalty_value=5))
            value = np.array(value)    #適應值

            dict = {"chromosome_0":population[:,0], 
                    "chromosome_1":population[:,1],
                    "chromosome_2":population[:,2],
                    "chromosome_3":population[:,3],
                    "chromosome_4":population[:,4],
                    "chromosome_5":population[:,5],
                    "chromosome_6":population[:,6],
                    "fitness value":value}
            df = pd.DataFrame(dict)
            sort_df = df.sort_values(by='fitness value', ascending = False)  #依照適應值排序

            population_selection = np.stack((sort_df["chromosome_0"].values[:int(len(sort_df)*selection_rate)], 
                                             sort_df["chromosome_1"].values[:int(len(sort_df)*selection_rate)], 
                                             sort_df["chromosome_2"].values[:int(len(sort_df)*selection_rate)], 
                                             sort_df["chromosome_3"].values[:int(len(sort_df)*selection_rate)], 
                                             sort_df["chromosome_4"].values[:int(len(sort_df)*selection_rate)], 
                                             sort_df["chromosome_5"].values[:int(len(sort_df)*selection_rate)],  
                                             sort_df["chromosome_6"].values[:int(len(sort_df)*selection_rate)]), axis=-1)

            population_crossover = population_selection.copy()  #直接複製

            c1type = False
            c2type = False
            while len(population_crossover) < population_size:
                for i, c in enumerate(population_selection):
                    if crossover_rate > random.uniform(0, 1):
                        if c1type:
                            c2 = population_selection[i]
                            c2type = True
                        else:
                            c1 = population_selection[i]
                            c1type = True

                        if c1type and c2type:
                            new_c1, new_c2 = crossover(c1, c2, point=2)
                            population_crossover = np.append(population_crossover,[new_c1],axis=0)
                            if len(population_crossover) >= population_size:
                                break
                            population_crossover = np.append(population_crossover,[new_c2],axis=0)
                            c1type = False
                            c2type = False
                            if len(population_crossover) >= population_size:
                                break
                        else:
                            pass
                if len(population_crossover) >= population_size:
                    break
            population_mutation = population_crossover.copy()

            for i, c in enumerate(population_mutation[int(population_size*selection_rate):], int(population_size*selection_rate)):  
                if mutation_rate > random.uniform(0, 1):
                    population_mutation[i] = mutation(c, point=1)

            population = population_mutation.copy()
            counter += 1
            text = "epoch:%5s,   fitness:%5s"%(str(counter), str(max(value)))
            self.textBrowser_show.append(text)
            
            
        value = []
        for p in population:
            value.append(fitness_function(chromosome=p, max_weight=150, data=self.article, penalty_value=5))
        value = np.array(value)    #適應值

        dict = {"chromosome_0":population[:,0], 
                "chromosome_1":population[:,1],
                "chromosome_2":population[:,2],
                "chromosome_3":population[:,3],
                "chromosome_4":population[:,4],
                "chromosome_5":population[:,5],
                "chromosome_6":population[:,6],
                "fitness value":value}
        df = pd.DataFrame(dict)
        sort_df = df.sort_values(by='fitness value', ascending = False)  #依照適應值排序
        text = str(sort_df.loc[0].values)
        self.textBrowser_show.append(text)
        

if __name__=="__main__":
    app = QApplication(sys.argv)
    win = MainWindow()
    win.show()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
